# 🕵️ Anomaly Detection Demo Notebook
Visualize results from our trained models.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import joblib
import pandas as pd

from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler


In [ ]:
# Load the data
df = pd.read_csv("../data/creditcard.csv")
X = df.drop(["Class", "Time"], axis=1, errors="ignore")
y = df["Class"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# Load trained models
iso_model = joblib.load("../models/isolation_forest.pkl")
ae_model = keras.models.load_model("../models/best_autoencoder_model.h5")
lstm_model = keras.models.load_model("../models/best_lstm_autoencoder_model.h5")


In [ ]:
# Predict with models
errors_ae = np.mean(np.square(X_scaled - ae_model.predict(X_scaled)), axis=1)
X_seq = X_scaled.reshape(-1, 1, X_scaled.shape[1])
errors_lstm = np.mean(np.square(X_seq - lstm_model.predict(X_seq)), axis=(1,2))
scores_if = -iso_model.decision_function(X_scaled)

# PR Curves
p1, r1, _ = precision_recall_curve(y, scores_if)
p2, r2, _ = precision_recall_curve(y, errors_ae)
p3, r3, _ = precision_recall_curve(y, errors_lstm)

plt.figure(figsize=(8,6))
plt.plot(r1, p1, label="Isolation Forest")
plt.plot(r2, p2, label="Autoencoder")
plt.plot(r3, p3, label="LSTM Autoencoder")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curves")
plt.legend()
plt.grid()
plt.show()


In [ ]:
from IPython.display import Image, display

display(Image("../reports/autoencoder_loss_curve.png"))
display(Image("../reports/lstm_autoencoder_loss_curve.png"))
display(Image("../reports/precision_recall_curves.png"))
